In [1]:
import numpy as np
from numba import jit

In [2]:
# initiates no_agents agents with no_outcomes uniform random evaluations of outcomes and returns them in a matrix.
def initiate_agents(no_agents, no_outcomes, group):
    
    # creates a matrix of agents with dimensionality N_A x N_O
    agent_eval = np.reshape(np.random.default_rng().uniform(0,1,no_agents*no_outcomes), (no_agents, no_outcomes))

    # every agent is assigned to either be part of the peer group or not with probability "group".
    group = np.array(np.random.choice([0,1], p=[(1-group),group], size=no_agents))
    
    return agent_eval, group

In [3]:
# calculates the attitude of a single agent
@jit(nopython=True)
def calculate_attitude_of_agent(evaluations, C):
    return np.dot(C, evaluations)

In [4]:
# calculates the attitude and norm of all agents.
@jit(nopython=True)
def calculate_attitude_of_agents(agents, C):
    return np.dot(agents, C.transpose())

In [5]:
# calculates the norm for every agent. The norm is equal for all external agents and all peer group member agents.
@jit(nopython=True)
def calculate_norm_of_agents(agent_norm=np.array([[]]), peer_norm=np.array([]), C=np.array([[]]), Society=np.array([]), agents_in_index=np.array([]), agents_out_index=np.array([]), gr_importance = 0):

    # the peer group norm is influenced by the peer norm and the societal norm.
    norm_in = np.dot(C, np.add((1-gr_importance) * Society, gr_importance * peer_norm))

    norm_out = np.dot(C, Society)

    # the norms are assigned to the respective indices
    agent_norm[agents_in_index,:] = norm_in
    agent_norm[agents_out_index,:] = norm_out

In [6]:
# calculates the norm of the peer group by calculating the median for each of the evaluations of outcomes of 
# group members.
def calculate_peer_norm(agents, agents_in_index):
    
    # gets the evaluations of all agents inside of the group.
    agents_in = agents[agents_in_index, :]
    
    # calculates the median for each evaluation
    peer_group_norm = np.median(agents_in, axis=0)

    return peer_group_norm

In [7]:
# calculates the evaluation adoption prefactor. Numbas jit is used for better performance.
@jit(nopython=True)
def calc_p_beta_diff(beta, coh_new, coh_old):
    return 1/(1+np.exp(beta * (coh_old - coh_new)))

# calculates the evaluation of the receiver based on the adoption factor and the sender's evaluation.
# numba is used for performance reasons.
@jit(nopython=True)
def calc_new_eval_rec(eval_newarg_before, p_beta_diff, eval_sen_arg):
    return eval_newarg_before * (1-p_beta_diff) + p_beta_diff * eval_sen_arg

In [8]:
# describes the argument evaluation exchange between sender and receiver. Currently, implements biased processing
# with the concept of cognitive coherence and sees the result as a prefactor for evaluation adoption.
@jit(nopython=True)
def simulate_interaction_on_individual_level_beh(eval_rec, eval_sen, att_rec, new_arg, beta, C):
    
    # calculates the coherence of the receiver before a new argument. It is split up do reduce the amount of matrix
    # multiplications. The coherence is the sum of all links (c ele C) multiplied by the links 
    # corresponding evaluation and attitude.

    coherence_base = np.dot(att_rec, C)
    #coherence_base = att_rec @ C

    coherence_old = np.dot(coherence_base, eval_rec)
    #coherence_old = coherence_base @ eval_rec
    
    eval_newarg_before = eval_rec[new_arg]
    eval_rec[new_arg] = eval_sen[new_arg]
    
    # calculates the coherence if the receiver would fully adopt the sender's evaluation.
    coherence_new = np.dot(coherence_base, eval_rec)
    #coherence_new = coherence_base @ eval_rec
    
    p_beta_diff = calc_p_beta_diff(beta, coherence_new, coherence_old)

    eval_rec[new_arg] = calc_new_eval_rec(eval_newarg_before, p_beta_diff, eval_sen[new_arg])

In [9]:
# simulates the interaction between two agents, who interact with each other
@jit(nopython=True)
def simulate_interaction(agents_eval, agents_att, group, beta, p, C, ind_rec, ind_sen, communicated_argument, agents_in_index, agents_out_index):
    # if the receiver is a group member, with probability p, he speaks with another group member. 
    if group[ind_rec]:
        if np.random.random() <= p:
            ind_sen = np.random.choice(agents_in_index)
        else:
            ind_sen = np.random.choice(agents_out_index)
            
    # the receiver adopts the senders evaluation to a certain degree.
    diff = simulate_interaction_on_individual_level_beh(agents_eval[ind_rec,:], agents_eval[ind_sen,:], agents_att[ind_rec,:],
                                                 communicated_argument, beta, C)
    
    # the new attitude of the receiver (after evaluation adoption) is calculated.
    agents_att[ind_rec] = calculate_attitude_of_agent(agents_eval[ind_rec,:], C)

In [10]:
# if all evaluations for a single outcome are within a certain threshold (epsilon) consens is assumed.
def check_for_same_eval(agents_eval, epsilon):
    # Calculate the range of values in each column
    col_ranges = np.ptp(agents_eval, axis=0)
    
    # Check if the maximum range in any column is less than epsilon
    consens = np.max(col_ranges) <= epsilon
    
    return consens

In [11]:
# simulates the whole model. If SyPaAn is True, only the state of the model after the last iteration will be returned.
def simulate_agent_interaction(no_of_agents, no_of_iterations, M, beta, group_prob, p,
                               compliance, C, Society, SyPaAn, check_for_consens):
    
    # Only if we are not conduction a Systematic Parameter Analysis will we need these lists
    if not SyPaAn:
        list_of_attitude_lists = []
        list_of_eval_lists = []
        norms = []
    
    # initiates the agents
    agents_eval, agents_group = initiate_agents(no_of_agents, M, group_prob)
    #calculates the initial peer group norm
    peer_norm = calculate_peer_norm(agents_eval, agents_group)
    
    # the indices of agents inside/outside of the group. For better performance it is only calculated once. 
    agents_in_index = np.where(agents_group > 0)[0]
    agents_out_index = np.where(agents_group < 1)[0]
    
    #calculates the initial norm and stores it in a N_A x N_B matrix for simpler access.
    agents_norm = np.zeros((no_of_agents, C.shape[0]))
    calculate_norm_of_agents(agents_norm, peer_norm, C, Society, agents_in_index, agents_out_index, p)
    
    # calculates initial attitude of all agents
    agents_att = calculate_attitude_of_agents(agents_eval, C)
    
    # to save computing time, the indices of the receivers and senders are drawn before the simulation 
    # for all iterations. The index of the sender might change later. The communicated argument is also determined
    # for faster runtime.
    #l_ind_rec, l_ind_sen, l_communicated_argument = create_random_indexes()
    l_ind_rec = np.random.randint(no_of_agents, size=no_of_iterations)
    l_ind_sen = np.random.randint(no_of_agents, size=no_of_iterations)
    l_communicated_argument = np.random.randint(M, size=no_of_iterations)
    
    # when the simulation is checked for consens, it is only checked once at the halftime point to prevent a loss of 
    # performance in every iteration.
    consens_check_at_halftime = (no_of_iterations/2)

    #simulates a single iteration
    ran = range(no_of_iterations)
    for i in ran:
        
        # simulates the interaction between two agents.
        simulate_interaction(agents_eval, agents_att, agents_group, beta, p, C,
                             l_ind_rec[i], l_ind_sen[i], l_communicated_argument[i], 
                             agents_in_index, agents_out_index)
        
        # if all evaluations are within a certain threshold, consens is assumed and the simulation stopped.
        if check_for_consens and (i == consens_check_at_halftime):
            if check_for_same_eval(agents_eval, 0.005):
                break
            
        # data about the simulation run is collected and stored for later analysis. It is only stored after a
        # "Macro-iteration", meaning after no_of_agents iteration.
        if (not SyPaAn) and (i%no_of_agents==0):
            # recalculates the peer norm 
            peer_norm = calculate_peer_norm(agents_eval, agents_in_index)
            norms.append(peer_norm)
            
            # recalculates the norm
            calculate_norm_of_agents(agents_norm, peer_norm, C, Society, agents_in_index, agents_out_index, p)
            
            list_of_eval_lists.append(agents_eval.copy())
            
            # calculates the final attitude towards the behaviours
            att_final = compliance * agents_norm + (1-compliance) * agents_att
            list_of_attitude_lists.append(att_final)
    
    # if a Systematic Parameter Analysis is performed, only the state of the agents 
    # after the last iteration is of concern
    if SyPaAn:
        # calculates peer norm and agent norm to allow the calculation of the final attitude
        peer_norm = calculate_peer_norm(agents_eval, agents_in_index)
        calculate_norm_of_agents(agents_norm, peer_norm, C, Society, agents_in_index, agents_out_index, p)
            
        att_final = compliance * agents_norm + (1 - compliance) * agents_att
        return att_final, agents_in_index
    
    return list_of_attitude_lists, norms, list_of_eval_lists, agents_in_index

In [15]:
params = {
        'no_of_agents': 100, 
        "no_of_iterations" : 100000,
        "M" : 3,

        "ß" : 13,
        "group_membership" : 0.2,
        "group_interaction": 0.2,
    
        "compliance" : 0,
    
        "C" : np.array([[1,-1,0],[-1,0,1]], dtype=np.float64),
        "Society" : np.array([1, 0, 1], dtype=np.float64),
    
        "SPA": False }

In [16]:
loal, norms, lovl, agents_in_index = simulate_agent_interaction(params["no_of_agents"], params["no_of_iterations"], params["M"], params["ß"], params["group_membership"], params["group_interaction"], params["compliance"], params["C"], params["Society"], params["SPA"], False)

%store params
%store loal
%store norms
%store lovl
%store agents_in_index

Stored 'params' (dict)
Stored 'loal' (list)
Stored 'norms' (list)
Stored 'lovl' (list)
Stored 'agents_in_index' (ndarray)


In [21]:
params = {
    'no_of_agents': 100, 
    "no_of_iterations" : 100000,
    "M" : 3,
    
    "ß" : 13,
    "group_membership" : 0.2,
    "group_interaction": 0.2,
    
    "compliance" : 0,
    
    "C" : np.array([[1,-1,0],[-1,0,1]], dtype=np.float64),
    "Society" : np.array([1, 0, 1], dtype=np.float64),
    
    "SPA": True}

SPA_param = {
    'params_to_iter': np.array(['ß','g','c']),
    'sims_per_comb': 50,
    'boundaries': np.array([[0,7],[0,1],[0,1]]),
    'no_of_steps': np.array([8,5,25])}

In [22]:
import itertools
# implements the iteration through a predefined parameter space
def systematic_parameter_analysis(SPA, params):

    # list which will contain the results
    results_in_matrix = []

    params_possbls = []
    # the parameter values that are iterated over are created using the upper and lower boundary provided by a variable, and the provided number of steps for each parameter
    for i in range(len(SPA['params_to_iter'])):
        params_possbls.append(np.linspace(SPA['boundaries'][i, 0], SPA['boundaries'][i, 1], SPA['no_of_steps'][i]))

    # creates the cartesion product out of the parameter values
    cartesian = itertools.product(*params_possbls)

    # runs a certain number of simulations for every parameter combination
    for ele in cartesian:
        print(np.round(ele,2))
        l_results_comb = []

        for i in range(SPA['sims_per_comb']):
            # runs the model and returns the attitudes after the last iteration, as well as the inidices of the group members
            results_comb, agents_in_index = simulate_agent_interaction(params["no_of_agents"], params["no_of_iterations"], params["M"], np.round(ele[0],2), params["group_membership"], np.round(ele[1],2), np.round(ele[2],2), params["C"], params["Society"], params["SPA"], True)
            l_results_comb.append(results_comb)

        # saves the results in a dictionary
        dict_comb = {'attitudes': l_results_comb, 'group_index': agents_in_index}

        # adds the parameter combination to the dictionary
        for i, ele_param in enumerate(SPA['params_to_iter']):
            dict_comb.update({ele_param: np.round(ele[i],2)})

        # adds the dictionary to the results list
        results_in_matrix.append(dict_comb)

    return results_in_matrix

In [23]:
# executes the systematic parameter analysis
result1 = systematic_parameter_analysis(SPA_param, params)

[0. 0. 0.]
[0.   0.   0.04]
[0.   0.   0.08]
[0.   0.   0.12]
[0.   0.   0.17]
[0.   0.   0.21]
[0.   0.   0.25]
[0.   0.   0.29]
[0.   0.   0.33]
[0.   0.   0.38]
[0.   0.   0.42]
[0.   0.   0.46]
[0.  0.  0.5]
[0.   0.   0.54]
[0.   0.   0.58]
[0.   0.   0.62]
[0.   0.   0.67]
[0.   0.   0.71]
[0.   0.   0.75]
[0.   0.   0.79]
[0.   0.   0.83]
[0.   0.   0.88]
[0.   0.   0.92]
[0.   0.   0.96]
[0. 0. 1.]
[0.   0.25 0.  ]
[0.   0.25 0.04]
[0.   0.25 0.08]
[0.   0.25 0.12]
[0.   0.25 0.17]
[0.   0.25 0.21]
[0.   0.25 0.25]
[0.   0.25 0.29]
[0.   0.25 0.33]
[0.   0.25 0.38]
[0.   0.25 0.42]
[0.   0.25 0.46]
[0.   0.25 0.5 ]
[0.   0.25 0.54]
[0.   0.25 0.58]
[0.   0.25 0.62]
[0.   0.25 0.67]
[0.   0.25 0.71]
[0.   0.25 0.75]
[0.   0.25 0.79]
[0.   0.25 0.83]
[0.   0.25 0.88]
[0.   0.25 0.92]
[0.   0.25 0.96]
[0.   0.25 1.  ]
[0.  0.5 0. ]
[0.   0.5  0.04]
[0.   0.5  0.08]
[0.   0.5  0.12]
[0.   0.5  0.17]
[0.   0.5  0.21]
[0.   0.5  0.25]
[0.   0.5  0.29]
[0.   0.5  0.33]
[0.   0.5  0.38

In [24]:
# stores the results for analysis in a different notebook
%store result1

Stored 'result1' (list)
